In [0]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot

In [0]:
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import os
from keras.models import load_model
from numpy import vstack
from matplotlib import pyplot as plt
from numpy.random import randint
import numpy as np
import cv2 as cv
from skimage.transform import resize, rescale

In [0]:
import pickle
from PIL import Image
from matplotlib import cm

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def read_data(path,x,y):
    x_train=[]
    count=0
    for i in range(10,950):
        image_file=path+x+'/'+str(i)+'.png'
        image_arr = plt.imread(image_file)
        # image_arr=image_arr[0:1050,0:1100]
#         image_arr= cv.resize(image_arr,(256,256))
        image_arr= image_arr[...,:3]
        # image_resized = resize(image_arr, (512, 512),anti_aliasing=True)
#         x_train.append(image_arr[0:1050,0:1100])
        x_train.append(image_arr)
        count+=1
        if(count%100==0):
           print(count)
          
    print("started for y")
    y_train=[]
    count=0
    for i in range(10,950):
        image_file=path+y+'/'+str(i)+'.png'
        image_arr = plt.imread(image_file)
        image_arr
        # image_arr=image_arr[0:1050,0:1100]
        # im = im.resize((256, 256), Image.ANTIALIAS)
        image_arr= image_arr[...,:3]
#         image_arr= cv.resize(image_arr,(256,256))
        image_resized = resize(image_arr, (256, 256),anti_aliasing=True)
        y_train.append(image_resized)
        count+=1
        if (count%100==0):
          print(count)
#         y_train.append(image_arr[0:1050,0:1100])
    
    return [asarray(x_train), asarray(y_train)] 
    
    
  
  
  

In [0]:
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model

In [0]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g
 
# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

In [0]:
# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512, dropout=False)
	d5 = decoder_block(d4, e3, 256, dropout=False)
	d6 = decoder_block(d5, e2, 128, dropout=False)
	d7 = decoder_block(d6, e1, 64, dropout=False)
	# output
	g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

In [0]:
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
	return model

In [0]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y
 
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y
 
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=3):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# scale all pixels from [-1,1] to [0,1]
	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0
	# plot real source images
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_realA[i])
	# plot generated target image
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + n_samples + i)
		pyplot.axis('off')
		pyplot.imshow(X_fakeB[i])
	# plot real target image
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
		pyplot.axis('off')
		pyplot.imshow(X_realB[i])
	# save plot to file
	filename1 = model_path+ 'plot_%06d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = model_path + 'model_%06d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

In [0]:
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
	# determine the output square shape of the discriminator
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
		if (i+1) % (bat_per_epo * 5) == 0:
			summarize_performance(i, g_model, dataset)

In [0]:
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
	images = vstack((src_img, gen_img, tar_img))
	# scale from [-1,1] to [0,1]
	images = (images + 1) / 2.0
	titles = ['Source', 'Generated', 'Expected']
	# plot images row by row
	for i in range(len(images)):
		# define subplot
		pyplot.subplot(1, 3, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(images[i])
		# show title
		pyplot.title(titles[i])
	pyplot.show()

In [0]:
len(dataset[0])

500

In [6]:
path= 'gdrive/My Drive/DS 5500/Data/PSdata/Google_Images/processed/'
dataset= read_data(path,'superimposed_950','street_950')

100
200
300
400
500
600
700
800
900
started for y
100
200
300
400
500
600
700
800
900


In [0]:
model_path='gdrive/My Drive/DS 5500/Data/PSdata/Google_Images/Gan_Model_5/'

In [0]:
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]
print(image_shape)

Loaded (940, 256, 256, 3) (940, 256, 256, 3)
(256, 256, 3)


In [0]:
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)

In [0]:
train(d_model, g_model, gan_model, dataset)

In [0]:
image_shape = x.shape[1:]

(512, 512, 3)

In [0]:
model_gen=load_model('gdrive/My Drive/DS 5500/Data/PSdata/Google_Images/Gan_Model_5/model_018800.h5')

In [0]:
model_gen

In [0]:
predicted=model_gen.predict(dataset[0])

In [9]:
np.shape(predicted)

(940, 256, 256, 3)

In [0]:
save_path='gdrive/My Drive/DS 5500/Data/PSdata/Google_Images/Gan_Model_5/predicted/'
plt.imshow(predicted[img])

In [0]:
save_path='gdrive/My Drive/DS 5500/Evaluation/New_gan_model/'
pickle.dump( dataset,save_path+"dataset",'wb')
pickle.dump( predicted,save_path+"predicted",'wb')


In [0]:
save_path='gdrive/My Drive/DS 5500/Evaluation/New_gan_model/'
with open(save_path+"dataset", 'wb') as file:
    pickle.dump(dataset,file)

In [0]:
save_path='gdrive/My Drive/DS 5500/Evaluation/New_gan_model/'
with open(save_path+"predicted", 'wb') as file:
    pickle.dump(predicted,file)

In [43]:
np.shape(dataset[1])

(940, 256, 256, 3)

In [0]:
model_gen=load_model('gdrive/My Drive/DS 5500/GAN Model_1000/model_010000.h5')

In [0]:
def read_data(path,x,y):
    x_train=[]
    count=0
    for i in range(10,510):
        image_file=path+x+'/'+str(i)+'.jpg'
        image_arr = plt.imread(image_file)
        image_arr=image_arr[0:1050,0:1100]
#         image_arr= cv.resize(image_arr,(256,256))
        image_arr= image_arr[...,:3]
        image_resized = resize(image_arr, (512, 512),anti_aliasing=True)
#         x_train.append(image_arr[0:1050,0:1100])
        x_train.append(image_resized)
        count+=1
        if(count%100==0):
           print(count)
          
    print("started for y")
    y_train=[]
    count=0
    for i in range(10,510):
        image_file=path+y+'/'+str(i)+'.jpg'
        image_arr = plt.imread(image_file)
        image_arr
        image_arr=image_arr[0:1050,0:1100]
        # im = im.resize((256, 256), Image.ANTIALIAS)
        image_arr= image_arr[...,:3]
#         image_arr= cv.resize(image_arr,(256,256))
        image_resized = resize(image_arr, (512, 512),anti_aliasing=True)
        y_train.append(image_resized)
        count+=1
        if (count%100==0):
          print(count)
#         y_train.append(image_arr[0:1050,0:1100])
    
    return [asarray(x_train), asarray(y_train)] 

In [7]:
path= 'gdrive/My Drive/DS 5500/Data/Google_Images/'
dataset= read_data(path,'superimposed_1000','street_1000')

100
200
300
400
500
started for y
100
200
300
400
500


In [0]:
np.shape(predicted)

In [0]:
predicted=model_gen.predict(dataset[0])

In [0]:
save_path='gdrive/My Drive/DS 5500/Evaluation/old_working/'
with open(save_path+"dataset", 'wb') as file:
    pickle.dump(dataset,file)

In [0]:
save_path='gdrive/My Drive/DS 5500/Evaluation/old_working/'
with open(save_path+"predicted", 'wb') as file:
    pickle.dump(predicted,file)

In [6]:
path= 'gdrive/My Drive/DS 5500/Evaluation/old_working/dataset'
predicted = open(path,'rb')
dataset_file = pickle.load(predicted)
predicted.close

<function BufferedReader.close>

In [0]:
im = Image.fromarray(np.uint8(cm.gist_earth(dataset_file[1][1])*255))

In [0]:
im = Image.fromarray(np.uint8(cm.gist_earth(dataset_file[1][1])*255)[...,:3])

(512, 512, 3, 3)